In [1]:
# Quintin B. Rozelle
# CS 340
# Project two
# 12/5/2023

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import CRUD module
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# database username and passord
username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div(children=[
    html.Div(children=[
        html.A(style={'float': 'left'}, href='https://www.snhu.edu', children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '100px'})
        ]),
        html.Center(
            html.B(
                html.H1('CS-340 Dashboard - Quintin B. Rozelle')
            )
        )
    ]),
    html.Hr(style={'clear': 'both'}),
    html.Div(
        
        # Code for the interactive filtering
        dcc.RadioItems(['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'],
                       'Reset',
                       inline=True,
                       id='filter_type')
    ),
    html.Hr(),
    
    # Filterable, sortable, pageable datatable
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',
                         filter_action='native',
                         sort_action='native',
                         sort_mode='multi',
                         selected_columns=[],
                         selected_rows=[0],
                         page_action='native',
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    
    # Charts
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Updates data table based on selected filter
@app.callback(Output('datatable-id','data'),
              [Input('filter_type', 'value')])
def update_dashboard(filter_type):
    
    # Water rescue
    if filter_type == 'Water Rescue':
        df = pd.DataFrame.from_records(db.read({
            '$or': [
                {'breed': 'Labrador Retriever Mix'},
                {'breed': 'Chesapeake Bay Retriever'},
                {'breed': 'Newfoundland'}
            ],
            'sex_upon_outcome': 'Intact Female',
            '$and': [
                {'age_upon_outcome_in_weeks': {'$gte': 26}},
                {'age_upon_outcome_in_weeks': {'$lte': 156}}
            ]
        }))
    
    # Mountain rescue
    elif filter_type == 'Mountain Rescue':
        df = pd.DataFrame.from_records(db.read({
            '$or': [
                {'breed': 'German Shepherd'},
                {'breed': 'Alaskan Malamute'},
                {'breed': 'Old English Sheepdog'},
                {'breed': 'Siberian Husky'},
                {'breed': 'Rottweiler'}
            ],
            'sex_upon_outcome': 'Intact Male',
            '$and': [
                {'age_upon_outcome_in_weeks': {'$gte': 26}},
                {'age_upon_outcome_in_weeks': {'$lte': 156}}
            ]
        }))
    
    # Disaster rescue
    elif filter_type == 'Disaster Rescue':
        df = pd.DataFrame.from_records(db.read({
            '$or': [
                {'breed': 'Doberman Pinscher'},
                {'breed': 'German Shepherd'},
                {'breed': 'Golden Retriever'},
                {'breed': 'Bloodhound'},
                {'breed': 'Rottweiler'}
            ],
            'sex_upon_outcome': 'Intact Male',
            '$and': [
                {'age_upon_outcome_in_weeks': {'$gte': 20}},
                {'age_upon_outcome_in_weeks': {'$lte': 300}}
            ]
        }))
    
    # Reset filter; gets entire database
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    # drop first column
    df.drop(columns=['_id'],inplace=True)
    
    return df.to_dict('records')
        

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    # Check to see that derived_virtual_data isn't empty. Do nothing if it is
    if viewData is None:
        return
    
    # If not emtpy, display counts of each breed in a histogram
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(figure=px.histogram(dff, x='breed', title='Available Animals'))
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    
    # check if any of the inputs are empty. If they are, do nothing
    if viewData is None:
        return
    elif index is None:
        return
    
    # If not empty, update geolocation map
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connection successful
Dash app running on http://127.0.0.1:12234/
